This script builds, trains, and evaluates a nueral network surrogate model to predict the deflection of a beam based on the generated dataset.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import time
import random
import os

In [ ]:
RANDOM_SEED = 42

os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # This is a key setting
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.config.experimental.enable_op_determinism()

In [213]:
# --- 1. Load Data ---
print("Loadind dataset...")
df = pd.read_csv('beam_deflection_dataset.csv')
print("Dataset loaded successfully.")
print("\nDataset Info:")
df.info()
print("\nFirst 5 rows of the dataset:")
print(df.info())

Loadind dataset...
Dataset loaded successfully.

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   k0        2000 non-null   float64
 1   k1        2000 non-null   float64
 2   damping   2000 non-null   int64  
 3   velocity  2000 non-null   float64
 4   w_max     2000 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 78.3 KB

First 5 rows of the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   k0        2000 non-null   float64
 1   k1        2000 non-null   float64
 2   damping   2000 non-null   int64  
 3   velocity  2000 non-null   float64
 4   w_max     2000 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 78.3 KB
None


In [214]:
df

,k0,k1,damping,velocity,w_max
0,1.288671e+07,491480.701340,0,52.865624,0.000544
1,1.773134e+07,327140.079848,0,71.830203,0.000618
2,4.383103e+07,76106.954349,0,73.764909,0.000192
3,2.274700e+06,161478.163969,0,63.154674,0.002299
4,8.889102e+06,246471.775806,0,16.989365,0.000437
...,...,...,...,...,...
1995,4.155442e+07,344917.038121,0,27.219681,0.000141
1996,1.237696e+07,414292.633413,0,22.143672,0.000412
1997,4.508789e+07,433046.020551,0,51.892469,0.000159
1998,1.097250e+07,381051.761320,0,76.768543,0.000889


In [215]:
df.head()

,k0,k1,damping,velocity,w_max
0,1.288671e+07,491480.701340,0,52.865624,0.000544
1,1.773134e+07,327140.079848,0,71.830203,0.000618
2,4.383103e+07,76106.954349,0,73.764909,0.000192
3,2.274700e+06,161478.163969,0,63.154674,0.002299
4,8.889102e+06,246471.775806,0,16.989365,0.000437


In [216]:
df.tail()

,k0,k1,damping,velocity,w_max
1995,4.155442e+07,344917.038121,0,27.219681,0.000141
1996,1.237696e+07,414292.633413,0,22.143672,0.000412
1997,4.508789e+07,433046.020551,0,51.892469,0.000159
1998,1.097250e+07,381051.761320,0,76.768543,0.000889
1999,3.624886e+07,377098.617153,0,23.084981,0.000173


In [217]:
# --- 2. Data Pre-processing ---

# Separate the inpute features (x) from the output target (y).
x = df[['k0', 'k1', 'damping', 'velocity']]
y = df['w_max']

In [218]:
# Split data into training + validation (85%) and testing (15%) sets.
x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

In [219]:
# Split the 85% block into training (70%) and validation (15%)
# The new test_size is 15/85 to get 15% of the original total data.
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=(0.15/0.85), random_state=42)

In [220]:
len(x_train)

1400

In [221]:
len(x_test)

300

In [222]:
# scale the input features
# We fit the scaler ONLY on the training data to prvent data leakage.
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)


In [223]:
print(F"\nData split complete (70/15/15).")
print(F"Total samples: {len(df)}")
print(F"Training samples: {len(x_train)}")
print(F"Validation samples: {len(x_val)}")
print(F"Testing samples: {len(x_test)}")


Data split complete (70/15/15).
Total samples: 2000
Training samples: 1400
Validation samples: 300
Testing samples: 300


In [224]:
# --- 3. Build the Neural Network Model ---
print("\nBuilding the Neural Network Model...")



Building the Neural Network Model...


In [225]:
# Input Layer: The shape must match the number of input features (4).
model = tf.keras.Sequential([tf.keras.layers.Input(shape=(x_train_scaled.shape[1],)),
                             
# Deeper and wider architecture
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2), # Add Dropout layer to prevent overfitting
    
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2), # Add another Dropout layer
    
    tf.keras.layers.Dense(64, activation='relu'),
    
    tf.keras.layers.Dense(1) # Output layer
])

# Use a lower learning rate for the Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

In [226]:
# Compile the model.
# We define the Optimizer (Adam), Loss function (Mean Squared Error), and any Metrics to track.
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [227]:
# Print a summary of the model architecture.
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 256)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,497 (166.00 KB)

 Trainable params: 42,497 (166.00 KB)

 Non-trainable params: 0 (0.00 B)

In [228]:
# --- 4. Train the Model ---
start_time = time.time()
print("\nTraining the model...")
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
history = model.fit(x_train_scaled, y_train, epochs=500, validation_data=(x_val_scaled, y_val),
                    batch_size=32, callbacks=[early_stopping], verbose=1)


Training the model...
Epoch 1/500


44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 6.8825e-04 - mae: 0.0194 - mse: 6.8825e-04 - val_loss: 1.3885e-05 - val_mae: 0.0029 - val_mse: 1.3885e-05
Epoch 2/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.1611e-05 - mae: 0.0060 - mse: 6.1611e-05 - val_loss: 8.5435e-06 - val_mae: 0.0023 - val_mse: 8.5435e-06
Epoch 3/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.4596e-05 - mae: 0.0043 - mse: 3.4596e-05 - val_loss: 2.9967e-06 - val_mae: 0.0011 - val_mse: 2.9967e-06
Epoch 4/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.2160e-05 - mae: 0.0035 - mse: 2.2160e-05 - val_loss: 4.1434e-06 - val_mae: 0.0013 - val_mse: 4.1434e-06
Epoch 5/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.3059e-05 - mae: 0.0027 - mse: 1.3059e-05 - val_loss: 3.0451e-06 - val_mae: 8.7451e-04 - val_mse: 3.0451e-06
Epoch 6/500
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.6434e-06 - mae: 0.0022 - mse: 9.6434e-06 - val_loss: 3.1802e-06 - val_mae: 0.0010 - val_mse: 3.1802e-06
Epoch 7/500
44/44

In [229]:
end_time = time.time()
training_time = end_time - start_time
print(F"\nTraining complete.")
print(F"\nTraining time: {training_time:.2f} seconds.")


Training complete.

Training time: 74.34 seconds.


In [230]:
# --- 5. Evaluate the Model ---
print(F"\nEvaluating the Model on the test set...")


Evaluating the Model on the test set...


In [231]:
# Make predictions on the unseen test data
y_pred = model.predict(x_test_scaled).flatten() # flatten converts a column vector

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [232]:
print(y_test.shape)
print(y_pred.shape)
print(x_test_scaled.shape)

(300,)
(300,)
(300, 4)


In [233]:
# Calculate performance metrics.

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

In [234]:
print(F"R2 Score: {r2:.4f}")
print(F"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(F"Mean Absolute Error (MAE): {mae:.4f}")

R2 Score: 0.9369
Root Mean Squared Error (RMSE): 0.0002
Mean Absolute Error (MAE): 0.0001
